Ejercicio 1

In [3]:
import numpy as np
import pandas as pd

def generar_dataset_ventas(n=500, seed=42):
    rng = np.random.default_rng(seed)

    # Catálogos
    productos = np.array(["Laptop", "Monitor", "Teclado", "Mouse", "Impresora", "Audífonos", "Webcam"])
    zonas = np.array(["Norte", "Centro", "Sur", "Oriente", "Poniente"])

    # Precio base por producto (para que el monto tenga sentido)
    precio_base = {
        "Laptop": 800_000,
        "Monitor": 180_000,
        "Teclado": 25_000,
        "Mouse": 18_000,
        "Impresora": 160_000,
        "Audífonos": 35_000,
        "Webcam": 45_000
    }

    # Variables principales
    producto = rng.choice(productos, size=n, p=[0.12, 0.18, 0.18, 0.18, 0.10, 0.14, 0.10])
    zona = rng.choice(zonas, size=n, p=[0.18, 0.28, 0.22, 0.18, 0.14])

    # Fechas (últimos 365 días)
    start = pd.Timestamp.today().normalize() - pd.Timedelta(days=365)
    fecha = start + pd.to_timedelta(rng.integers(0, 366, size=n), unit="D")

    # Cantidad (más realista: muchos 1-3, pocos grandes)
    cantidad = rng.choice([1, 2, 3, 4, 5, 6, 8, 10], size=n, p=[0.35, 0.25, 0.16, 0.08, 0.06, 0.04, 0.04, 0.02])

    # Precio unitario con variación por producto (ruido + pequeño efecto por zona)
    base = np.array([precio_base[p] for p in producto], dtype=float)

    # Efecto por zona (solo para generar patrones)
    factor_zona = {
        "Norte": 0.98,
        "Centro": 1.00,
        "Sur": 0.97,
        "Oriente": 1.05,
        "Poniente": 0.99
    }
    fz = np.array([factor_zona[z] for z in zona], dtype=float)

    # Variación aleatoria del precio (normal, recortada para no explotar)
    ruido = rng.normal(loc=1.0, scale=0.08, size=n)
    ruido = np.clip(ruido, 0.85, 1.25)

    precio_unitario = np.round(base * fz * ruido, 0)

    # Monto total
    monto = np.round(precio_unitario * cantidad, 0)

    # Armar DataFrame
    df = pd.DataFrame({
        "producto": producto,
        "monto": monto.astype(int),
        "cantidad": cantidad.astype(int),
        "fecha": fecha,
        "zona": zona,
        "precio_unitario": precio_unitario.astype(int)
    })

    # (Opcional) meter algunos valores faltantes y outliers para EDA
    # 2% nulos en monto
    idx_nulos = rng.choice(df.index, size=max(1, int(n * 0.02)), replace=False)
    df.loc[idx_nulos, "monto"] = np.nan

    # 1% outliers en monto (ventas extremadamente altas)
    idx_out = rng.choice(df.dropna().index, size=max(1, int(n * 0.01)), replace=False)
    df.loc[idx_out, "monto"] = (df.loc[idx_out, "monto"] * rng.integers(6, 12, size=len(idx_out))).astype(int)

    return df

# --- Generar y guardar ---
df = generar_dataset_ventas(n=500, seed=42)
print(df.head())

df.to_csv("data\ventas_simuladas.csv", index=False, encoding="utf-8")
print("\nArchivo guardado: ventas_simuladas.csv")

    producto     monto  cantidad      fecha      zona  precio_unitario
0  Audífonos   37514.0         1 2025-12-17   Oriente            37514
1    Teclado   26779.0         1 2025-02-21  Poniente            26779
2  Audífonos   34196.0         1 2025-06-24  Poniente            34196
3  Impresora  443961.0         3 2025-07-16       Sur           147987
4     Laptop  659600.0         1 2025-11-18       Sur           659600

Archivo guardado: ventas_simuladas.csv
